## Intro to Web Scraping in Python

Today we will learn how to scrape HTML web pages in python, using the Beautiful Soup 4 library. We can programmatically gather information from websites to use for your own purposes. We will gather information about the UVA basketball team, first by walking through each step of the process. Then by doing it in a more automated way.

First we need to install the Beautiful Soup 4 and lxml libraries. These are not a part of base python or the Anaconda installation.

In [1]:
!conda install --yes beautifulsoup4
!conda install --yes lxml
!conda install --yes requests

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::r-base64enc==0.1_3=r36h46e59ec_4
  - defaults/noarch::r-dbplyr==1.4.0=r36h6115d3f_0
  - defaults/osx-64::r-xml2==1.2.0=r36h466af19_0
  - defaults/noarch::r-bh==1.69.0_1=r36h6115d3f_0
  - defaults/osx-64::argon2-cffi==20.1.0=py37haf1e3a3_1
  - defaults/osx-64::r-bit64==0.9_7=r36h46e59ec_0
  - defaults/osx-64::r-askpass==1.0=r36h1de35cc_0
  - defaults/osx-64::jupyter==1.0.0=py37_7
  - defaults/noarch::r-xfun==0.6=r36h6115d3f_0
  - defaults/osx-64::r-bitops==1.0_6=r36h46e59ec_4
  - defaults/osx-64::r-yaml==2.2.0=r36h46e59ec_0
  - defaults/osx-64::bcrypt==3.2.0=py37haf1e3a3_0
  - defaults/osx-64::r-base==3.6.1=hcb44179_1
  - defaults/osx-64::widgetsnbextension==3.5.1=py37_0
  - defaults/noarch::r-generics==0.0.2=r36h6115d3f_0
  - defaults/noarch::numpydoc==1.1.0=py_0
  - defaults/osx-64::notebook==6.1.1=py37_0
  - defau

Next, let's import the libraries we will be using in this Jupyter Notebook, including Beautiful Soup 4. The other two, requests and pandas, are already installed if you are using Anaconda.

In [2]:
import requests
import lxml
from bs4 import BeautifulSoup
import pandas as pd


Now we are ready to make an HTTP request using the requests library. This means that once you have established a connection with the destination (the server which hosts the website you want to communicate with), the client (you) sends an HTTP GET request to the server to retrieve the website and all data within it. 

This is typically done by your web browser, but we can also do it in python. 

Today we are going to scrape information about players on the UVA men's basketball team. You can find the team's roster for the 2021-2022 season here: https://virginiasports.com/sports/mbball/roster/ 

We will start by gathering information about one of UVA's players, Kihei Clark.

In [3]:
#This is how to make an HTTP GET request using the requests library.
source = requests.get('https://virginiasports.com/sports/mbball/roster/season/2021-22/player/kihei-clark/')

print(source)      #this prints the type of response. 200 means "OK". There are many response codes

<Response [200]>


Now, we have a response from the server. If the response is good, the source code of the web page is contained within that response. Let's see what that looks like

In [ ]:
soup = BeautifulSoup(source.text, 'lxml')

print(soup) 


This is messy, but it is all the code for the page we have issued a request for. Some of it is human readable, some of it is not. Now, let's look at the source code of this page another way. Copy and paste this link into your web browser: https://virginiasports.com/sports/mbball/roster/season/2021-22/player/kihei-clark/

### Note - You need to use Google Chrome to have access to inspector and other developer tools

Right click somewhere on your page and click "inspect". Then, make sure to choose the 'elements' tab to see the HTML source code of this page. While inspecting the page elements, you can see which parts of the page are controlled by different parts of the code. Notice that the code starts with large chunks (< body > for example), and has divisions within that (< div > tags), among others.

The class "bio-info" looks like it contains the majority of the information in the body of the page. Let's start with this. We are going to scrape some information about Kihei Clark from this page.

In [ ]:
# the prettify() function makes the code somewhat more readable.
# I don't use this feature much but maybe you will appreciate it.
print(soup.prettify())

The find() function finds the first item matching this criteria. Notice our arguments are first, the HTML tag, and second, the class within that tag.

In [5]:
player_info = soup.find("div", class_='bio-info')    #class_, because 'class' is a reserved word in python
print(player_info)

<div class="bio-info">
<div class="text-block">
<h1>Kihei Clark</h1>
<div class="info-block">
<div>
<div class="value">Guard</div>
<div class="description">Position</div>
</div>
<div>
<div class="value">5'10''</div>
<div class="description">Height</div>
</div>
<div>
<div class="value">172 lbs.</div>
<div class="description">Weight</div>
</div>
<div>
<div class="value">Senior</div>
<div class="description">Class</div>
</div>
<div>
<div class="value">Woodland Hills, Calif.</div>
<div class="description">Hometown</div>
</div>
<div>
<div class="value">Taft Charter</div>
<div class="description">High School / Club</div>
</div>
<div>
<div class="value"><a href="https://twitter.com/ClarkKihei" rel="nofollow" target="_blank"><i class="fab fa-twitter"></i> @ClarkKihei</a></div>
<div class="description">Twitter</div>
</div>
<div>
<div class="value"><a href="https://instagram.com/kihei.clark/" rel="nofollow" target="_blank"><i class="fab fa-instagram"></i> @kihei.clark</a></div>
<div class="descr

I now see that the player's name is inside a couple more tags inside that 'bio-info' class. Let's drill down into the code and get the player's name value.

In [6]:
player_name = soup.find('div', class_='bio-info').div.h1.text
print(player_name)


Kihei Clark


Next, I want more information about the player such as his position, height, weight, etc. 

Looking again at the HTML, it looks like all of that information is in repetitive 'div' tags. Let's select all of those elements.

In [9]:
for item in soup.find_all('div', class_="value"):
    print(item.text)

['Guard', "5'10''", '172 lbs.', 'Senior', 'Woodland Hills, Calif.', 'Taft Charter', ' @ClarkKihei', ' @kihei.clark']


I also want to get the labels for this data. This would be things such as 'position', 'height', 'weight', etc. 

In [10]:
for item in soup.find_all('div', class_='description'):
    print(item.text)

Position
Height
Weight
Class
Hometown
High School / Club
Twitter
Instagram


Lets do a little more here just to clean things up. In the following cell I will do the same as the previous two cells, but this time I will store the data in a list. Then I will zip those lists together and make a nice dictionary with my data.

In [11]:
player_stats=[]
labels=[]

for i in soup.find_all('div', class_='value'):
    player_stats.append(i.text)
    
for i in soup.find_all('div', class_='description'):
    labels.append(i.text)
    
#let's assume I only want the first five items in each list
player_stats = player_stats[:5]
labels = labels[:5]

player_dict = dict(zip(labels, player_stats))
print(player_dict)

{'Position': 'Guard', 'Height': "5'10''", 'Weight': '172 lbs.', 'Class': 'Senior', 'Hometown': 'Woodland Hills, Calif.'}


Now I have an easy to use dictionary!

In [12]:
print(player_dict['Weight'])

172 lbs.


# Selenium

Now that we have learned to scrape static HTML content, let's automate this task using Selenium. Check out the Selenium documentation here: https://www.selenium.dev/

### Selenium Web Driver
In order to use Selenium, we must download and install a web driver which allows you to drive a browser with your code.

**Important**
The following code assumes you are using Google Chrome and will use the associated web driver. If you are using another browser (safari, firefox, edge, etc) you will need to download the selenium web driver for that browser. Just check the Selenium documentation in order to do that.

You also need to make sure to download the correct web driver which corresponds to your version of Google Chrome.

I have included a detailed writeup about this in the 'WebDriverInstall.md' file in the github repository.
    

In [13]:
# install selenium
!conda install --yes selenium

Solving environment: done

# All requested packages already installed.



In [14]:
# Import selenium and webdrivers

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time

Let's make sure now that Selenium is working and you have all your paths set up correctly. **Important**: You will have to change the path below to the path of 'driver' on your own computer.

If this works correctly, it will open up a blank browser with a message that 'this is being controlled by automated test software'

In [18]:
# Make sure that Selenium is working and you have all your paths set up correctly
driver = webdriver.Chrome(executable_path="/Users/ep9k/Desktop/PythonWebScraping-master/chromedriver")

Now we can go directly to a page of our choice like so...

In [19]:
driver = webdriver.Chrome(executable_path="/Users/ep9k/Desktop/PythonWebScraping-master/chromedriver")
driver.get('https://virginiasports.com/sports/mbball/roster/')

We can now proceed to write out script just like any other program. Just like BeautifulSoup, Selenium provides the ability to select HTML elements by the tag name, class name, id name, and so on. 

On the UVA roster homepage, looking at the HTML you can see that each player has it's own box with a picture, name, and so on. The HTML is the same for all of those players. In our example, we will simulate that the user is clicking on each player to see that player's individual page. Then we will scrape the information off of that page just like we did in earler. Let's start with just one player.

In [20]:
driver = webdriver.Chrome(executable_path="/Users/ep9k/Desktop/PythonWebScraping-master/chromedriver")
driver.get('https://virginiasports.com/sports/mbball/roster/')

#I am selecting by x path.
player = driver.find_element_by_xpath('//*[@id="players"]/div[1]/div[2]/div[1]')
player.click()

See above that I used the 'find_element_by_xpath' method to click on this player's name. Let's define what an XPath is.

**XPath**: XPath enables testers to navigate through the XML structure of an HTML or XML document. Don't worry too much about this. I like to use 'find_element_by_xpath' because it is basically a unique identifier for items in the HTML document and makes selecting them easy.

There are many different ways to select HTML elements in Selenium. Check out the documentation for more examples: https://selenium-python.readthedocs.io/locating-elements.html

Now that we have Selenium installed and set up and we did a small example, we can now expand upon it. We will take just a few players from the team and iterate through them. Once we land on each page we will do exactly what we just did with static HTML with BeautifulSoup.

In [22]:
driver = webdriver.Chrome(executable_path="/Users/ep9k/Desktop/PythonWebScraping-master/chromedriver")
driver.get('https://virginiasports.com/sports/mbball/roster/')


#we will use a while loop to grab data about the first three players
count = 0

while count < 3:
    count += 1
    time.sleep(1)
    
    #select player's name using find_element_by_xpath()
    #each player's name is a link to their bio page
    player = driver.find_element_by_xpath(f'//*[@id="players"]/div[{count}]/div[2]/div[1]')
    player.click()
    
    #now we use beautiful soup to parse the HTML just as we did last time
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    player_name = soup.find('div', class_='bio-info').div.h1.text
    print()
    print(player_name)
    
    for item in soup.find_all('div', class_="value"):
        print(item.text) 
        
    #go back to roster page after collecting information about current player    
    driver.get('https://virginiasports.com/sports/mbball/roster/')
    
driver.quit()






Kihei Clark
Guard
5'10''
172 lbs.
Senior
Woodland Hills, Calif.
Taft Charter
 @ClarkKihei
 @kihei.clark

Jayden Gardner
Forward
6'6''
246 lbs.
Senior
Wake Forest, N.C.
Heritage
East Carolina
 @Jayy_Baller_1
 @ssjjaygee

Reece  Beekman
Guard
6'2''
181 lbs.
Sophomore
Baton Rouge, La.
Scotlandville Magnet
 @reece_beekman
 @reece.2


Let's do just a little more to make it pretty. This time we will collect the information about each player and put it into a pandas dataframe. There are many ways to do this but I will use lists to populate the columns of the dataframe.

In [23]:
df = pd.DataFrame(columns=['Name', 'Position', 'Height', 'Weight', 'Year'])
print(df)

Empty DataFrame
Columns: [Name, Position, Height, Weight, Year]
Index: []


We will do the whole thing together.

In [24]:

driver = webdriver.Chrome(executable_path="/Users/ep9k/Desktop/PythonWebScraping-master/chromedriver")
driver.get('https://virginiasports.com/sports/mbball/roster/')

#accumulator lists we will use later to make our pandas dataframe
names = []
positions = []
heights = []
weights = []
years = []


#we will use a while loop to grab data about the first three players
count = 0

team_players = ['Kihei Clark', 'Jayden Gardner', 'Reece Beekman', 'Armaan Franklin',
               'Jayden Nixon', 'Taine Murray', 'Malachi Poindexter', 'Chase Coleman',
               'Kadin Shedrick', 'Francisco Caffaro', 'Kody Stattman', 'Igor Milicic Jr',
               'Carson McCorkle']

#there are 13 players on the team
for i in team_players:
    count += 1
    time.sleep(1)
    
    #select player's name using find_element_by_xpath()
    #each player's name is a link to their bio page
    player = driver.find_element_by_xpath(f'//*[@id="players"]/div[{count}]/div[2]/div[1]')
    player.click()
    
    #now we use beautiful soup to parse the HTML just as we did last time
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    #player info will be used to store the data about each player in a list
    player_info = []
    
    for item in soup.find_all('div', class_="value"):
        player_info.append(item.text)
        
    #add the information from player_info to the accumulator lists outside this loop
    names.append(i)    # i is player name from team_players
    positions.append(player_info[0])
    heights.append(player_info[1])
    weights.append(player_info[2])
    years.append(player_info[3])
        
    #go back to roster page after collecting information about current player    
    driver.get('https://virginiasports.com/sports/mbball/roster/')
    
driver.quit()


Lastly, we will take the lists we created above and put them into the pandas dataframe we created earlier

In [25]:
df['Name'] = names
df['Position'] = positions
df['Height'] = heights
df['Weight'] = weights
df['Year'] = years
print(df)

                  Name       Position  Height    Weight                Year
0          Kihei Clark          Guard  5'10''  172 lbs.              Senior
1       Jayden Gardner        Forward   6'6''  246 lbs.              Senior
2        Reece Beekman          Guard   6'2''  181 lbs.           Sophomore
3      Armaan Franklin          Guard   6'4''  204 lbs.              Junior
4         Jayden Nixon          Guard   6'3''  200 lbs.              Senior
5         Taine Murray          Guard   6'5''  207 lbs.            Freshman
6   Malachi Poindexter          Guard   6'2''  190 lbs.           Sophomore
7        Chase Coleman          Guard   5'9''  161 lbs.              Junior
8       Kadin Shedrick        Forward  6'11''  231 lbs.  Redshirt Sophomore
9    Francisco Caffaro         Center   7'1''  242 lbs.     Redshirt Junior
10       Kody Stattman          Guard   6'8''  200 lbs.              Senior
11     Igor Milicic Jr  Guard/Forward  6'10''  224 lbs.            Freshman
12     Carso